# Sisendtekstide valideerimine ja puhastamine

[![License: BSD-2-Clause](https://img.shields.io/badge/License-BSD--2--Clause-lightgrey.svg)](https://opensource.org/license/bsd-2-clause/)


Tekstide puhastamiseks ja valideerimiseks on mitmeid viise. 
Üks lihtsamaid võimalusi on kõigepealt teha tekstides standardsed asendused ning seejärel kontrollida tulemust vaadates sõnastuse tulemusena tekkivate sõnavormide sagedust ning otsides neist võimalikke erindeid.  


In [1]:
import re
import math
import time
import requests
import datetime
import urllib.parse

from datetime import date
from pandas import DataFrame 
from pandas import read_csv
from pandas import concat
from tqdm.auto import tqdm

## I. Tekstidele rakendatavad standardsed normaliseerimisvõtted

Asendame kõik standardsed sõnaeraldajad tühikuga ning eraldame kirjavahemärgid mõlemalt poolt tühikuga.
Standardsed loomuliku keele töötluse vahendid nagu EstNLTK teevad veelgi rohkem selliseid teisendusi. 
Meie eesmärk siin on teha need asendused lihtsustamaks teksti, et muude vigade tuvastamine oleks lihtsam.  

In [2]:
sources = {}
sources['state_laws'] = read_csv('../results/source_texts/state_laws.csv', header=0)
sources['government_regulations'] = read_csv('../results/source_texts/government_regulations.csv', header=0)
sources['local_government_acts'] = read_csv('../results/source_texts/local_government_acts.csv', header=0)
sources['government_orders'] = read_csv('../results/source_texts/government_orders.csv', header=0)

In [3]:
PUNCTUATION_MARK = '[\.,:;!?]'
PUNCTUATION_MARK_GROUP = f'(?P<mark>{PUNCTUATION_MARK})'
PUNCTUATION_MARK_REPLACEMENT = r' \g<mark> '
assert re.sub(PUNCTUATION_MARK_GROUP, PUNCTUATION_MARK_REPLACEMENT, 'male,kabe') == 'male , kabe'

In [4]:
for name, tbl in sources.items(): 
    tbl['document_title'] = tbl['document_title'].str.replace(PUNCTUATION_MARK_GROUP, PUNCTUATION_MARK_REPLACEMENT, regex=True)
    tbl['document_title'] = tbl['document_title'].str.replace('\s+', ' ', regex=True)

## II. Vabamorfi kasutamine sõnade algvormi leidmiseks

Tekstide edasisel analüüsimisel saame kasutada Vabamorfi veebiteenust, mis leiab igale sõnale vastava algvormi.
Kuna mõnele sõnavormile vastab mitu erinevat algvormi (`sadama` -> `sadam`, `sadama`) siis on oluline teada, kas
Vabamorf kasutab kõrvalolevate sõnade konteksti tuvastamaks, milline neist võimalikest algvormidest on õige.
Kuna see protsess pole alati veavaba, siis antud analüüsis kasutame kõiki sõnale vastavaid algvorme.
Vabamorfi teiseks eripäraks on oletamine. Vabamorf kasutab sisemiselt suurt sõnastikku algvormide määramiseks.
Kui oletamine on sisse lülitatud, siis kasutatakse heuristilisi reegleid algvormide leidmiseks sõnadele, mis pole suures sõnastikus.
Oletamise kasutamine on suurte tekstide analüüsimisel vajalik, et saaks korrektselt käsitleda erialatermineid ja slängi. 

**Tehniline märkus.** Sõnade lemmatiseerimiseks saab kasutada kahte erinevat veebiteenust 
* sõnade morfoloogilist analüsaatorit (`analyser`)
* dokumentide indekseerijat (`api_advanced_indexing`)

Neist esimene analüüsib sisendit sõnade haaval ning teine analüüsib kogu sisendit korraga ning ühestab mitmesed analüüsid lähtuvalt kontekstist. Antud juhul sobib meile sõnahaaval tehtud analüüs, kuna me valideerime sisendi korrektsust.  

In [5]:
def analyze_text(caption: str):
    """
    Uses web service to extract words and sub-words form document captions

    Returns a four column table with columns index, wordform, lemma, sublemmas.
    There can be several rows for each word as each word is analysed separately.
    All rows with the same index correspond to the same word.
    Wordform columns is added to facilitate tokenisation debugging.
    """
    ANALYZER_QUERY = "https://smart-search.tartunlp.ai/api/analyser/process"
    HEADERS = {"Content-Type": "application/json; charset=utf-8"}
    POST_DATA_TEMPLATE = {'params': {"vmetajson": ["--guess"]}, 'content': caption}

    response = requests.post(ANALYZER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"
    response = response.json()

    token_count = len(response['annotations']['tokens'])
    tbl = DataFrame({'wordform': [None] * token_count, 'lemma': [None] * token_count})
    for i, token in enumerate(response['annotations']['tokens']):
        features = token['features']
        tbl.loc[i, 'wordform'] = features['token']
        tbl.loc[i, 'lemma'] = list(set(map(lambda x: x['lemma'], features['mrf'])))

    tbl =  tbl.reset_index().explode('lemma')

    # Post-correction for Vabamorph output. Remove special symbols 
    tbl['lemma'] = tbl['lemma'].str.replace('=', '', regex=False)
    tbl['lemma'] = tbl['lemma'].str.replace('+', '', regex=False)

    # Post-correction for sublemmas
    tbl['sublemmas'] = tbl['lemma'].str.split('_', regex=False)
    tbl['lemma'] = tbl['lemma'].str.replace('_', '', regex=False)
    return tbl

# Example output
analyze_text('Presidendi ametiraha seadus')

,index,wordform,lemma,sublemmas
0,0,Presidendi,president,[president]
1,1,ametiraha,ametiraha,"[ameti, raha]"
2,2,seadus,seadu,[seadu]
2,2,seadus,seadus,[seadus]


In [6]:
all_documents = concat([source for _,source in sources.items()], axis=0)

In [7]:
result = [None] *  len(all_documents)
for i, caption in tqdm(enumerate(all_documents['document_title']), total=len(all_documents)):
    result[i] = analyze_text(caption).assign(doc_id = i)

result = concat(result, axis=0)
display(result)

  0%|          | 0/58837 [00:00<?, ?it/s]

,index,wordform,lemma,sublemmas,doc_id
0,0,Ajutise,ajutine,[ajutine],0
1,1,sisseveo,sissevedu,"[sisse, vedu]",0
2,2,konventsiooniga,konventsioon,[konventsioon],0
3,3,ühinemise,ühinemine,[ühinemine],0
4,4,seadus,seadu,[seadu],0
...,...,...,...,...,...
0,0,Jõe,jõgi,[jõgi],58836
1,1,paadisadama,paadisadam,"[paadi, sadam]",58836
2,2,akvatooriumi,akvatoorium,[akvatoorium],58836
3,3,piiride,piir,[piir],58836


## III. Sisendtekstide analüüs

Kõige lihtsam on anomaalilad tuvastada analüüsides esmalt sõna algvorme (lemmasid).
Kõigepealt on kasulik otsida anomaaliaid harva esinevate algvormide hulgast.

In [8]:
lemma_counts = (result
                .groupby('lemma').agg(occurence_count=('lemma', len), document_count = ('doc_id', lambda x: len(set(x))))
                .sort_values(['occurence_count', 'document_count'], ascending=False).reset_index()
                .pipe(lambda df: df[df['lemma'].str.len() > 0])
               )
display(lemma_counts.tail(15))

,lemma,occurence_count,document_count
16256,üüritariif,1,1
16257,üüritoetus,1,1
16258,üüriturg,1,1
16259,üürivaidlus,1,1
16260,ˮEuroopa,1,1
16261,ˮinimõigus,1,1
16262,ˮsisekokkulepe,1,1
16263,ˮüks,1,1
16264,‎tingimus,1,1
16265,’),1,1


**Tulemus:** Harva esinevate lemmade hulgas on tõesti palju anomaalseid sõnu. 

In [9]:
ESTONIAN_LETTER = '[a-z|öäõü|\-|žš]'
ESTONIAN_LETTERS_ONLY = f'^(?:{ESTONIAN_LETTER})+$' 

Vaatame neid erandlikke sõnu lähemalt jättes alles kõik need sõnad, mis sisaldavad eesti tähestikku mitte kuuluvaid sümboleid.

In [10]:
incorrect = ~lemma_counts['lemma'].str.contains(ESTONIAN_LETTERS_ONLY, case=False, regex=True)
display(concat([
    lemma_counts[incorrect]
    .head(15)
    .reset_index(drop=True),
    lemma_counts[incorrect]
    .tail(15)
    .reset_index(drop=True)], axis=1))

,lemma,occurence_count,document_count,lemma,occurence_count,document_count
0,",",7017,4385,tingimused1,1,1
1,.,4975,4153,vormid1,1,1
2,2017,784,780,¿9,1,1
3,2020,524,519,Äriseadustik1,1,1
4,2019,459,455,ˮEuroopa,1,1
5,2021,453,450,ˮinimõigus,1,1
6,2018,447,446,ˮsisekokkulepe,1,1
7,2022,428,424,ˮüks,1,1
8,2023,340,338,‎tingimus,1,1
9,1,215,196,’),1,1


**Tulemus:** Lemmade hulgas on palju spetsiifilisi sõnu: kirjavahemärke ja aastanumbreid.


In [11]:
NUMBER_SYMBOLS = '[0-9]'
NUMBERS_ONLY = f'^{NUMBER_SYMBOLS}+$'
numbers = lemma_counts['lemma'].str.contains(NUMBERS_ONLY, case=False, regex=True)
unanalysed = incorrect & ~ numbers
display(lemma_counts[unanalysed & lemma_counts['lemma'].str.contains(f'{NUMBER_SYMBOLS}+')])

,lemma,occurence_count,document_count
512,2014-2020,133,128
578,2016-2019,112,112
605,2017-2020,107,107
616,COVID-19,105,105
625,2014-2017,104,103
...,...,...,...
16073,¿9,1,1
16078,Äriseadustik1,1,1
16268,−2025,1,1
16269,−2026,1,1


**Tulemus:** Lisaks numbritele on meil ka numbrivahemikud

In [12]:
DASH_SYMBOLS = '[-−‒]'
SLASH_SYMBOLS = '[/]'
NUMBER = f'{NUMBER_SYMBOLS}+'
NUMBER_RANGE = f'(?:(?:{NUMBER})?{DASH_SYMBOLS}{NUMBER})|(?:{NUMBER}{DASH_SYMBOLS})|(?:{NUMBER}{SLASH_SYMBOLS}{NUMBER})'
NUMBER_RANGE_ONLY = f'^(?:{NUMBER_RANGE})$'
number_range = lemma_counts['lemma'].str.contains(NUMBER_RANGE_ONLY)
assert re.match(NUMBER_RANGE_ONLY, '-2012') is not None
assert re.match(NUMBER_RANGE_ONLY, '1930-2023') is not None
assert re.match(NUMBER_RANGE_ONLY, '2017/2018') is not None
assert re.match(NUMBER_RANGE_ONLY, '−2025') is not None
assert re.match(NUMBER_RANGE_ONLY, '2017-') is not None
assert re.match(NUMBER_RANGE_ONLY, '2014‒2020') is not None
unanalysed = incorrect & ~ numbers & ~number_range 
display(lemma_counts[unanalysed & lemma_counts['lemma'].str.contains(NUMBER)])

,lemma,occurence_count,document_count
616,COVID-19,105,105
4898,SARS-CoV-2,5,5
5070,kord1,5,5
6381,2014-2020ˮ,3,3
6401,5%,3,3
...,...,...,...
14602,rakendamine1,1,1
15449,tingimused1,1,1
15969,vormid1,1,1
16073,¿9,1,1


**Tulemus:** Lisaks numbrivahemikele on olemas ka protsendid ja kirjavahemärgid

In [13]:
PERCENT_SYMBOLS ='[%]'
PERCENT_ONLY = f'^(?:{NUMBER}{PERCENT_SYMBOLS})$'
PUNCTUATION_MARK_ONLY =f'^(?:{PUNCTUATION_MARK})$'
percents = lemma_counts['lemma'].str.contains(PERCENT_ONLY)
punctuation_marks = lemma_counts['lemma'].str.contains(PUNCTUATION_MARK_ONLY)
unanalysed = incorrect & ~ numbers & ~number_range & ~percents & ~punctuation_marks 
display(lemma_counts[unanalysed & lemma_counts['lemma'].str.contains(NUMBER)])

,lemma,occurence_count,document_count
616,COVID-19,105,105
4898,SARS-CoV-2,5,5
5070,kord1,5,5
6381,2014-2020ˮ,3,3
6504,NO99,3,3
...,...,...,...
14602,rakendamine1,1,1
15449,tingimused1,1,1
15969,vormid1,1,1
16073,¿9,1,1


**Tulemus:** Jutumärgid on jäänud kirjavahemärkide hulgast välja.

In [14]:
PUNCTUATION_MARK = '[\.,:;!?"ˮ“\)\(]'
PUNCTUATION_MARK_GROUP = f'(?P<mark>{PUNCTUATION_MARK})'
PUNCTUATION_MARK_REPLACEMENT = r' \g<mark> '
assert re.sub(PUNCTUATION_MARK_GROUP, PUNCTUATION_MARK_REPLACEMENT, 'male,kabe') == 'male , kabe'
assert re.sub(PUNCTUATION_MARK_GROUP, PUNCTUATION_MARK_REPLACEMENT, '2010ˮ') == '2010 ˮ '
assert re.sub(PUNCTUATION_MARK_GROUP, PUNCTUATION_MARK_REPLACEMENT, '(2010)') == ' ( 2010 ) '
punctuation = lemma_counts['lemma'].str.contains(f'^{PUNCTUATION_MARK}$')
display(lemma_counts[punctuation].style.set_caption('Kirjavahemärgid'))
unanalysed &= ~punctuation
display(lemma_counts[unanalysed].head().style.set_caption('Veel liigitamata sõnad'))
display(lemma_counts[unanalysed & (lemma_counts['lemma'].str.len() == 1)].style.set_caption('Ühetähelised liigitamata sõnad'))

,lemma,occurence_count,document_count
4,",",7017,4385
8,.,4975,4153
3244,:,10,10
4757,;,6,5
10312,"""",1,1
16266,“,1,1


,lemma,occurence_count,document_count
616,COVID-19,105,105
716,§,90,75
3292,ja/või,10,10
4260,teedele/tänav,7,7
4831,/,5,5


,lemma,occurence_count,document_count
716,§,90,75
4831,/,5,5
16267,−,1,1


**Tulemus:** Lisaks punktuatsioonile on veel sümboleid, mida peaks ignoreerima.

Vaatame nüüd kahetähelisi ja kolmetähelisi sõnu, sest need sisaldavad ilmsesti ootamatuid sümboleid.  

In [15]:
OTHER_SYMBOLS = '[§/−a]'
OTHER_SYMBOLS_ONLY=f'^(?:{OTHER_SYMBOLS})$'
other_symbols = lemma_counts['lemma'].str.contains(OTHER_SYMBOLS_ONLY)
unanalysed &= ~ other_symbols
display(lemma_counts[unanalysed].head().style.set_caption('Veel liigitamata sõnad'))
display(lemma_counts[unanalysed & (lemma_counts['lemma'].str.len() == 2)].style.set_caption('Kahetähelised liigitamata sõnad'))
display(lemma_counts[unanalysed & (lemma_counts['lemma'].str.len() == 3)].style.set_caption('Kolmetähelised liigitamata sõnad'))

,lemma,occurence_count,document_count
616,COVID-19,105,105
3292,ja/või,10,10
4260,teedele/tänav,7,7
4898,SARS-CoV-2,5,5
5070,kord1,5,5


,lemma,occurence_count,document_count
7812,3¹,2,2
10386,1a,1,1
10543,3M,1,1
10563,4a,1,1
10564,4⁴,1,1
10639,9c,1,1
11086,L7,1,1
14000,m²,1,1
16073,¿9,1,1
16265,’),1,1


,lemma,occurence_count,document_count
7809,23¹,2,2
7817,4S2,2,2
10334,10а,1,1
10352,13¹,1,1
10358,16¹,1,1
10367,18a,1,1
10387,2-k,1,1
10504,22¹,1,1
10506,23²,1,1
10509,24²,1,1


**Tulemus:** Spetsiaalsed ülaindeksite sümboleid tähistavad sümbolid on problemaatilised.

Uurime seda täpsemalt filtreerides välja kõik sõnad, milles olevaid sümboleid me juba lubame.

In [16]:
ALLOWED_SYMBOLS = f'{ESTONIAN_LETTER}|{NUMBER_SYMBOLS}|{PUNCTUATION_MARK}|{DASH_SYMBOLS}|{SLASH_SYMBOLS}|{PERCENT_SYMBOLS}|{OTHER_SYMBOLS}'
lemma_counts[unanalysed & ~lemma_counts['lemma'].str.contains(f'^(?:{ALLOWED_SYMBOLS})+$', regex=True, case=False)]

,lemma,occurence_count,document_count
6422,Côte,3,3
6469,Kohtla‑järv,3,3
6632,d’Ivoire,3,3
7809,23¹,2,2
7812,3¹,2,2
7858,Brasília,2,2
10334,10а,1,1
10352,13¹,1,1
10358,16¹,1,1
10410,2004-2006»meede,1,1


**Tulemus:** Eestikeele tähestikku tuleb veel lisada paar tähte katmaks ära võõrkeelseid nimesid.

In [17]:
ESTONIAN_LETTER = '[a-z|öäõü|\\-|žš|ôíë]'
ALLOWED_SYMBOLS = f'{ESTONIAN_LETTER}|{NUMBER_SYMBOLS}|{PUNCTUATION_MARK}|{DASH_SYMBOLS}|{SLASH_SYMBOLS}|{PERCENT_SYMBOLS}|{OTHER_SYMBOLS}'
lemma_counts[unanalysed & ~lemma_counts['lemma'].str.contains(f'^(?:{ALLOWED_SYMBOLS})+$', regex=True, case=False)]

,lemma,occurence_count,document_count
6469,Kohtla‑järv,3,3
6632,d’Ivoire,3,3
7809,23¹,2,2
7812,3¹,2,2
10334,10а,1,1
10352,13¹,1,1
10358,16¹,1,1
10410,2004-2006»meede,1,1
10499,2035”heakskiitmine,1,1
10504,22¹,1,1


**Tulemus:** Paljudes sõnades on unicode superskripti sümbolid. 

In [18]:
SUBSCRIPT_SYMBOLS = '[₀₁₂₃₄₅₆₇₈₉₊₋]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
lemma_counts[lemma_counts['lemma'].str.contains(f'{SUPERSCRIPT_SYMBOLS}|{SUBSCRIPT_SYMBOLS}', regex=True)]

,lemma,occurence_count,document_count
7809,23¹,2,2
7812,3¹,2,2
10352,13¹,1,1
10358,16¹,1,1
10504,22¹,1,1
10506,23²,1,1
10509,24²,1,1
10559,46⁷,1,1
10564,4⁴,1,1
10601,67³,1,1


In [19]:
ALLOWED_SYMBOLS = (f'{ESTONIAN_LETTER}|{NUMBER_SYMBOLS}|{PUNCTUATION_MARK}|{DASH_SYMBOLS}' 
                  f'|{SLASH_SYMBOLS}|{PERCENT_SYMBOLS}|{OTHER_SYMBOLS}|{SUPERSCRIPT_SYMBOLS}|{SUBSCRIPT_SYMBOLS}')
lemma_counts[unanalysed & ~lemma_counts['lemma'].str.contains(f'^(?:{ALLOWED_SYMBOLS})+$', regex=True, case=False)]

,lemma,occurence_count,document_count
6469,Kohtla‑järv,3,3
6632,d’Ivoire,3,3
10334,10а,1,1
10410,2004-2006»meede,1,1
10499,2035”heakskiitmine,1,1
10505,23^4,1,1
13000,kohta﻿,1,1
15266,taotlemine”dokumendivorm,1,1
16073,¿9,1,1
16264,‎tingimus,1,1


Leiame nüüd ülesse kõik sümbolid, mis jäävad lubatud sümbolitest välja.

In [20]:
idx = unanalysed & ~lemma_counts['lemma'].str.contains(f'^(?:{ALLOWED_SYMBOLS})+$', regex=True, case=False)
set.union(*lemma_counts.loc[idx, 'lemma'].map(lambda x: set(x) - set(re.findall(ALLOWED_SYMBOLS, x, flags=re.IGNORECASE))))

{'^', '»', '¿', 'а', '\u200e', '‑', '’', '”', '\ufeff'}

* Siit on näha, et me peame uuendama oma kirjavahemärkide definitsiooni lisades sinna erinevaid jutumärke. 
* Kuna ülakomasid kasutatakse käändelõppude eraldamiseks ja me eraldame kirjavahemärgid tühikutega, siis neid me kirjavahemärkide  hulka ei pane.
* Lisaks tuleb meil uuendada kriipsude sümboleid ning defineerida erikujuliste tühikute symbolid ning täiendada eestikeelset tähestikku.

In [21]:
DASH_SYMBOLS = '[‑-−‒]'
PUNCTUATION_MARK = '[\\.,:;!?¿\\(\\)«»„““ˮ"‟”]'
WHITESPACE_SYMBOLS = '[\u200e\ufeff]'
OTHER_SYMBOLS = '[§/−a\^]'
ESTONIAN_LETTER = '[a-z|öäõü|\\-|žš|ôíëа]'

ALLOWED_SYMBOLS = (f'{ESTONIAN_LETTER}|{NUMBER_SYMBOLS}|{PUNCTUATION_MARK}|{DASH_SYMBOLS}' 
                  f'|{SLASH_SYMBOLS}|{PERCENT_SYMBOLS}|{OTHER_SYMBOLS}|{SUPERSCRIPT_SYMBOLS}|{SUBSCRIPT_SYMBOLS}|{WHITESPACE_SYMBOLS}')
lemma_counts[unanalysed & ~lemma_counts['lemma'].str.contains(f'^(?:{ALLOWED_SYMBOLS})+$', regex=True, case=False)]

,lemma,occurence_count,document_count


## III. Sisendtekstide puhastamine

Eelnevast analüüsist lähtuvalt saame me nüüd defineerida sisendtekstide puhastamise skriptid, mis parandavad esialgset naiivset tokenisatsiooni:
* kõik tühikusümbolid ühestatakse;
* kirjavahemärgid ning muud erisümbolid eraldatakse mõlemalt poolt tühikutega;
* numbrivahemikkude korral eraldame kriipsu numbridümbolitest.

In [22]:
ESTONIAN_LETTER = '[a-z|öäõüžš]'
FOREIGN_LETTER = '[ôíëаa]'
NUMBER_SYMBOLS = '[0-9]'

# Tühikutega eraldatavad sümbolid 
PUNCTUATION_MARK = '[\\.,:;!?¿\\(\\)«»„““ˮ"‟”]'
SPECIAL_SYMBOLS = '[§/%\^]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
SUBSCRIPT_SYMBOLS = '[₀₁₂₃₄₅₆₇₈₉₊₋]'

# Tühikutega asendatavad sümbolid  
WHITESPACE_SYMBOLS = '[\u200e\ufeff]'

# Teised lubatud sümbolid
DASH_SYMBOLS = '[‑-−‒-]'
OTHER_SYMBOLS = '[§/−a\^]'

In [23]:
SEPARATION_GROUP = f'(?P<mark>{PUNCTUATION_MARK}|{SPECIAL_SYMBOLS}|{SUBSCRIPT_SYMBOLS}|{SUPERSCRIPT_SYMBOLS})'
SEPARATION_GROUP_REPLACEMENT = r' \g<mark> '
assert re.sub(SEPARATION_GROUP, SEPARATION_GROUP_REPLACEMENT, 'male,kabe') == 'male , kabe'
assert re.sub(SEPARATION_GROUP, SEPARATION_GROUP_REPLACEMENT, 'male/kabe') == 'male / kabe'
assert re.sub(SEPARATION_GROUP, SEPARATION_GROUP_REPLACEMENT, 'male¹') == 'male ¹ '
assert re.sub(SEPARATION_GROUP, SEPARATION_GROUP_REPLACEMENT, 'male₁') == 'male ₁ '

DASH_GROUP = f'(?P<head>^|\s|{WHITESPACE_SYMBOLS}|{NUMBER_SYMBOLS})(?P<dash>{DASH_SYMBOLS})(?P<tail>\s|{WHITESPACE_SYMBOLS}|{NUMBER_SYMBOLS}|$)'
DASH_GROUP_REPLACEMENT = r'\g<head> \g<dash> \g<tail>'
assert re.sub(DASH_GROUP, DASH_GROUP_REPLACEMENT, '1‑2') == '1 ‑ 2'
assert re.sub(DASH_GROUP, DASH_GROUP_REPLACEMENT, '-2021') == ' - 2021'
assert re.sub(DASH_GROUP, DASH_GROUP_REPLACEMENT, '2021- ') == '2021 -  '

WHITESPACE_GROUP = f'(?:\s|{WHITESPACE_SYMBOLS})+'
assert re.sub(WHITESPACE_GROUP, ' ', '  ') == ' '
assert re.sub(WHITESPACE_GROUP, ' ', '\t  ') == ' '
assert re.sub(WHITESPACE_GROUP, ' ', '\u200e\ufeff ') == ' '

In [24]:
sources = {}
sources['state_laws'] = read_csv('../results/source_texts/state_laws.csv', header=0)
sources['government_regulations'] = read_csv('../results/source_texts/government_regulations.csv', header=0)
sources['local_government_acts'] = read_csv('../results/source_texts/local_government_acts.csv', header=0)
sources['government_orders'] = read_csv('../results/source_texts/government_orders.csv', header=0)

In [25]:
for name, tbl in sources.items(): 
    tbl['document_title'] = tbl['document_title'].str.replace(SEPARATION_GROUP, SEPARATION_GROUP_REPLACEMENT, regex=True)
    tbl['document_title'] = tbl['document_title'].str.replace(DASH_GROUP, DASH_GROUP_REPLACEMENT, regex=True)
    tbl['document_title'] = tbl['document_title'].str.replace(WHITESPACE_GROUP, ' ', regex=True)
    tbl.to_csv(f'../results/cleaned_texts/{name}.csv')

## IV. Puhastatud sisendtekstide valideerimine

In [26]:
sources = {}
sources['state_laws'] = read_csv('../results/cleaned_texts/state_laws.csv', header=0)
sources['government_regulations'] = read_csv('../results/cleaned_texts/government_regulations.csv', header=0)
sources['local_government_acts'] = read_csv('../results/cleaned_texts/local_government_acts.csv', header=0)
sources['government_orders'] = read_csv('../results/cleaned_texts//government_orders.csv', header=0)

In [27]:
all_documents = concat([source for _,source in sources.items()], axis=0)

In [28]:
result = [None] *  len(all_documents)
for i, caption in tqdm(enumerate(all_documents['document_title']), total=len(all_documents)):
    result[i] = analyze_text(caption).assign(doc_id = i)

result = concat(result, axis=0)
display(result)

  0%|          | 0/58837 [00:00<?, ?it/s]

,index,wordform,lemma,sublemmas,doc_id
0,0,Ajutise,ajutine,[ajutine],0
1,1,sisseveo,sissevedu,"[sisse, vedu]",0
2,2,konventsiooniga,konventsioon,[konventsioon],0
3,3,ühinemise,ühinemine,[ühinemine],0
4,4,seadus,seadu,[seadu],0
...,...,...,...,...,...
0,0,Jõe,jõgi,[jõgi],58836
1,1,paadisadama,paadisadam,"[paadi, sadam]",58836
2,2,akvatooriumi,akvatoorium,[akvatoorium],58836
3,3,piiride,piir,[piir],58836


In [29]:
lemma_counts = (result
                .groupby('lemma').agg(occurence_count=('lemma', len), document_count = ('doc_id', lambda x: len(set(x))))
                .sort_values(['occurence_count', 'document_count'], ascending=False).reset_index()
                .pipe(lambda df: df[df['lemma'].str.len() > 0])
               )
display(lemma_counts.tail(15))

,lemma,occurence_count,document_count
15871,ümberarvestustegur,1,1
15872,ümberjaotav,1,1
15873,ümberlaadimiskoha,1,1
15874,ümberlaadimiskoht,1,1
15875,ümberregistreerimata,1,1
15876,ümberregistreerimine,1,1
15877,ümbertegemine,1,1
15878,üürikorter,1,1
15879,üüritariif,1,1
15880,üüritoetus,1,1


**Kontroll:** Üheelemendiliste sümbolite hulgas pole tundmatuid sümboleid.

In [30]:
idx = lemma_counts['lemma'].str.len() == 1
idx &= ~lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{NUMBER_SYMBOLS})$', case=False)
idx &= ~lemma_counts['lemma'].str.contains(f'^(?:{PUNCTUATION_MARK}|{SUBSCRIPT_SYMBOLS}|{SUPERSCRIPT_SYMBOLS})$', case=False)
idx &= ~lemma_counts['lemma'].str.contains(f'^(?:{DASH_SYMBOLS}|{SPECIAL_SYMBOLS})$', case=False)
assert not any(idx)

**Filtreerimine:** Eemaldame kõik normaalsete tähetedega lemmad ja numbrid ja õhesümbolised sõnad.

In [31]:
unanalysed = lemma_counts['lemma'].str.len() != 1

idx = lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER})+$', case=False)
unanalysed &= ~ idx
display(lemma_counts[idx].sample(n=10))

idx = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER_SYMBOLS})+$', case=False)
unanalysed &= ~ idx
display(lemma_counts[idx].sample(n=10))

,lemma,occurence_count,document_count
13580,mooniõis,1,1
2354,tehnika,17,17
1133,Ambla,50,50
2735,audit,14,10
5456,automatiseeritu,4,4
11822,esmaabialane,1,1
2629,aastane,14,14
5147,tasakaalustatud,5,5
8049,ee,2,2
15839,üldistatud,1,1


,lemma,occurence_count,document_count
3175,24,10,10
385,2,188,165
10120,181,1,1
10213,52,1,1
10170,332,1,1
10274,931,1,1
3631,09,8,8
10097,112,1,1
10251,73,1,1
10175,35,1,1


**Filtreerimine:** Eemaldame kõik sidekriipsuga sõnad

In [32]:
idx = lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{DASH_SYMBOLS})+$', case=False)
display(lemma_counts[idx & unanalysed].sample(10))
unanalysed &= ~idx

,lemma,occurence_count,document_count
12489,kesk-ordoviitsium,1,1
9677,tänava-ja,2,2
10551,Ida-Harju,1,1
10890,Nõva-Kullamaa,1,1
9920,HIV-nakkus,2,1
7786,Pihla-Kaibald,2,2
399,sise-eeskiri,181,181
10444,Eesti-Soome,1,1
13913,pedagoog-metoodik,1,1
4440,kambriumi-vent,6,6


**Tulemus:** Alles jääb hulk vigaseid sõnu ja lühendeid, mida tokenisatsioon ei peagi parandama:
* superscripti sümbol on lisatud nimisõnale;
* tekstile järgneb sidekriipsuga number;
* valesti kirjutatud sõnad tüüpi 17-aastane;
* lühendid kujul 104-k;
* lühendid kujul 10a ja 9c.

In [33]:
pattern = f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{DASH_SYMBOLS})*(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}){NUMBER_SYMBOLS}$'
idx = lemma_counts['lemma'].str.contains(pattern, case=False)
display(lemma_counts[idx])
unanalysed &= ~ idx

pattern = f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER})+{DASH_SYMBOLS}(?:{NUMBER_SYMBOLS})+$'
idx = lemma_counts['lemma'].str.contains(pattern, case=False)
display(lemma_counts[idx])
unanalysed &= ~ idx

pattern = f'^(?:{NUMBER_SYMBOLS})+(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER})*ne$'
idx = lemma_counts['lemma'].str.contains(pattern, case=False)
display(lemma_counts[idx])
unanalysed &= ~idx

pattern = f'^(?:{NUMBER_SYMBOLS})+{DASH_SYMBOLS}{ESTONIAN_LETTER}$'
idx = lemma_counts['lemma'].str.contains(pattern, case=False)
display(lemma_counts[idx])
unanalysed &= ~idx

pattern = f'^(?:{NUMBER_SYMBOLS})+{ESTONIAN_LETTER}$'
idx = lemma_counts['lemma'].str.contains(pattern, case=False)
display(lemma_counts[idx])
unanalysed &= ~idx

,lemma,occurence_count,document_count
4955,kord1,5,5
10386,CO2,1,1
10389,CT1,1,1
10390,CT2,1,1
10729,L7,1,1
11643,eeskiri1,1,1
12564,kinnitamine1,1,1
12633,kohta1,1,1
12894,kvaliteedinõuded1,1,1
12994,ladustamisega1,1,1


,lemma,occurence_count,document_count
635,COVID-19,105,105
11118,TKNE-5,1,1
11119,TKNE-7,1,1
11204,Valga-1,1,1
11206,Valka-2,1,1
15701,Äriseadustik1,1,1


,lemma,occurence_count,document_count
6217,19aastane,3,3
6226,7aastane,3,3
7564,10kroonine,2,2
7565,12meetrine,2,2
7574,24meetrine,2,2
7585,500kroonine,2,2
7588,5aastane,2,2
10086,1000ruutmeetrine,1,1
10087,100kroonine,1,1
10155,25kroonine,1,1


,lemma,occurence_count,document_count
7563,104-k,2,2
7590,625-k,2,2
10085,10-k,1,1
10088,1016-k,1,1
10089,105-k,1,1
...,...,...,...
10268,90-k,1,1
10269,902-k,1,1
10271,927-k,1,1
10276,94-k,1,1


,lemma,occurence_count,document_count
10126,18a,1,1
10140,1a,1,1
10166,313a,1,1
10184,3M,1,1
10190,43A,1,1
10205,4a,1,1
10261,80s,1,1
10281,9c,1,1


**Tulemus:** Allesjäänud lühendid ja sõnad on valiidsed.

In [34]:
lemma_counts[unanalysed]

,lemma,occurence_count,document_count
4782,SARS-CoV-2,5,5
6326,NO99,3,3
7584,4S2,2,2
10092,10а,1,1
10142,20-se,1,1
10204,4S3,1,1
10283,A1-alamkategooria,1,1
10351,B1-alamkategooria,1,1
10379,C1-alamkategooria,1,1
10388,CR14,1,1
